The following tutorials are available from the [Wallaroo Tutorials Repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/notebooks_in_prod).

# Stage 3: Deploy the Model in Wallaroo
 
In this stage, we upload the trained model and the processing steps to Wallaroo, then set up and deploy the inference pipeline. 

Once deployed we can feed the newest batch of data to the pipeline, do the inferences and write the results to a results table.

For clarity in this demo, we have split the training/upload task into two notebooks:

* `02_automated_training_process.ipynb`: Train and pickle ML model.
* `03_deploy_model.ipynb`: Upload the model to Wallaroo and deploy into a pipeline.

Assuming no changes are made to the structure of the model, these two notebooks, or a script based on them, can then be scheduled to run on a regular basis, to refresh the model with more recent training data and update the inference pipeline.

This notebook is expected to run within the Wallaroo instance's Jupyter Hub service to provide access to all required Wallaroo libraries and functionality.

## Resources

The following resources are used as part of this tutorial:

* **data**
  * `data/seattle_housing_col_description.txt`: Describes the columns used as part data analysis.
  * `data/seattle_housing.csv`: Sample data of the Seattle, Washington housing market between 2014 and 2015.
* **code**
  * `postprocess.py`: Formats the data after inference by the model is complete.
  * `simdb.py`: A simulated database to demonstrate sending and receiving queries.
  * `wallaroo_client.py`: Additional methods used with the Wallaroo instance to create workspaces, etc.
* **models**
  * `housing_model_xgb.onnx`: Model created in Stage 2: Training Process Automation Setup.

## Steps

The process of uploading the model to Wallaroo follows these steps:

* [Connect to Wallaroo](#connect-to-wallaroo): Connect to the Wallaroo instance and set up the workspace.
* [Upload The Model](#upload-the-model): Upload the model and autoconvert for use in the Wallaroo engine.
* [Upload the Processing Modules](#upload-the-processing-modules): Upload the processing modules.
* [Create and Deploy the Pipeline](#create-and-deploy-the-pipeline): Create the pipeline with the model and processing modules as steps, then deploy it.
* [Test the Pipeline](#test-the-pipeline): Verify that the pipeline works with the sample data.

### Connect to Wallaroo

First we import the required libraries to connect to the Wallaroo instance, then connect to the Wallaroo instance.

In [1]:
import json
import pickle
import pandas as pd
import numpy as np
import pyarrow as pa

import simdb # module for the purpose of this demo to simulate pulling data from a database

# from wallaroo.ModelConversion import ConvertXGBoostArgs, ModelConversionSource, ModelConversionInputType
import wallaroo
from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import datetime

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

wl = wallaroo.Client()

wallarooPrefix = "doc-test."
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")


In [3]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

In [4]:
model_name = "housepricemodel"
model_file = "./housing_model_xgb.onnx"
pipeline_name = "housing-pipe"

### Upload The Model

With the connection set and workspace prepared, upload the model created in `02_automated_training_process.ipynb` into the current workspace.

To ensure the model input contract matches the provided input, the configuration `tensor_fields=["tensor"]` is used so regardless of what the model input type is, Wallaroo will ensure inputs of type `tensor` are accepted.

In [5]:
hpmodel = (wl.upload_model(model_name, 
                           model_file, 
                           framework=wallaroo.framework.Framework.ONNX)
                           .configure(tensor_fields=["tensor"]
                                    )
            )

## Upload the Processing Modules

Preprocess model:

In [6]:
input_schema = pa.schema([
    pa.field('id', pa.int64()),
    pa.field('date', pa.string()),
    pa.field('list_price', pa.float64()),
    pa.field('bedrooms', pa.int64()),
    pa.field('bathrooms', pa.float64()),
    pa.field('sqft_living', pa.int64()),
    pa.field('sqft_lot', pa.int64()),
    pa.field('floors', pa.float64()),
    pa.field('waterfront', pa.int64()),
    pa.field('view', pa.int64()),
    pa.field('condition', pa.int64()),
    pa.field('grade', pa.int64()),
    pa.field('sqft_above', pa.int64()),
    pa.field('sqft_basement', pa.int64()),
    pa.field('yr_built', pa.int64()),
    pa.field('yr_renovated', pa.int64()),
    pa.field('zipcode', pa.int64()),
    pa.field('lat', pa.float64()),
    pa.field('long', pa.float64()),
    pa.field('sqft_living15', pa.int64()),
    pa.field('sqft_lot15', pa.int64()),
    pa.field('sale_price', pa.float64())
])

output_schema = pa.schema([
    pa.field('tensor', pa.list_(pa.float32(), list_size=18))
])

preprocess_model = wl.upload_model("preprocess-step", "./models/preprocess_step.zip", \
                                   framework=wallaroo.framework.Framework.PYTHON, \
                                   input_schema=input_schema, output_schema=output_schema)

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a container runtime..
Model is attempting loading to a container runtime........successful

Ready


Postprocess model:

In [7]:
input_schema = pa.schema([
    pa.field('variable', pa.list_(pa.float32()))
])


output_schema = pa.schema([
    pa.field('variable', pa.list_(pa.float32()))
])

postprocess_model = wl.upload_model("postprocess-step", "./models/postprocess_step.zip", \
                                   framework=wallaroo.framework.Framework.PYTHON, \
                                   input_schema=input_schema, output_schema=output_schema)

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a container runtime..
Model is attempting loading to a container runtime.......successful

Ready


### Create and Deploy the Pipeline

Create the pipeline with the preprocess module, housing model, and postprocess module as pipeline steps, then deploy the newpipeline.

In [8]:
pipeline = get_pipeline(pipeline_name)
# clear if the tutorial was run before
pipeline.clear()

pipeline.add_model_step(preprocess_model)
pipeline.add_model_step(hpmodel)
pipeline.add_model_step(postprocess_model)

name,housing-pipe
created,2024-03-14 21:40:49.410206+00:00
last_updated,2024-03-15 14:30:21.730581+00:00
deployed,True
arch,None
accel,None
tags,
versions,"2f010e0a-d8a4-432b-9bc0-e0aa6e99fa83, c233aaf7-565b-449b-b3bc-1ac099679663, a930f821-31aa-4adb-a29a-33044bb69259, b8d12b5c-27f7-4985-a6ba-e6c53f549018, 3c6a77cf-296d-433a-874d-0267ce544c11"
steps,preprocess-step
published,True


In [15]:
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

name,housing-pipe
created,2024-03-14 21:40:49.410206+00:00
last_updated,2024-03-15 14:32:01.571423+00:00
deployed,True
arch,None
accel,None
tags,
versions,"247bc22d-a702-4d04-9642-4a7766407b96, 26cfb381-ecae-465b-a9c5-15a459a0993f, 2f010e0a-d8a4-432b-9bc0-e0aa6e99fa83, c233aaf7-565b-449b-b3bc-1ac099679663, a930f821-31aa-4adb-a29a-33044bb69259, b8d12b5c-27f7-4985-a6ba-e6c53f549018, 3c6a77cf-296d-433a-874d-0267ce544c11"
steps,preprocess-step
published,True


In [9]:
pipeline

name,housing-pipe
created,2024-03-14 21:40:49.410206+00:00
last_updated,2024-03-15 14:30:21.730581+00:00
deployed,True
arch,None
accel,None
tags,
versions,"2f010e0a-d8a4-432b-9bc0-e0aa6e99fa83, c233aaf7-565b-449b-b3bc-1ac099679663, a930f821-31aa-4adb-a29a-33044bb69259, b8d12b5c-27f7-4985-a6ba-e6c53f549018, 3c6a77cf-296d-433a-874d-0267ce544c11"
steps,preprocess-step
published,True


In [10]:
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.28.1.176',
   'name': 'engine-6d44cb79b5-vpvd8',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'housing-pipe',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'config': {'batch_config': None,
       'filter_threshold': None,
       'id': 7,
       'input_schema': None,
       'model_version_id': 4,
       'output_schema': None,
       'runtime': 'onnx',
       'sidekick_uri': None,
       'tensor_fields': ['tensor']},
      'model_version': {'conversion': {'framework': 'onnx',
        'python_version': '3.8',
        'requirements': []},
       'file_info': {'file_name': 'housing_model_xgb.onnx',
        'sha': 'd8b79e526eed180d39d4653b39bebd9d06e6ae7f68293b5745775a9093a3ae7d',
        'version': '19323f59-37b8-47fa-978d-bf03f107a958'},
       'id': 4,
       'image_path': None,
       'name': 'housepricemodel',
       'status': 'ready',
       'tas

### Test the Pipeline

We will use a single query from the simulated `housing_price` table and infer.  When successful, we will undeploy the pipeline to restore the resources back to the Kubernetes environment.

In [11]:
conn = simdb.simulate_db_connection()

# create the query
query = f"select * from {simdb.tablename} limit 1"
print(query)

# read in the data
singleton = pd.read_sql_query(query, conn)
conn.close()

display(singleton.loc[:, ["id", "date", "list_price", "bedrooms", "bathrooms", "sqft_living", "sqft_lot"]])

select * from house_listings limit 1


,id,date,list_price,bedrooms,bathrooms,sqft_living,sqft_lot
0,7129300520,2023-08-02,221900.0,3,1.0,1180,5650


In [18]:
# inference via API call

import requests

# Retrieve the token
headers = wl.auth.auth_header()

# set Content-Type type
headers['Content-Type']='application/json; format=pandas-records'

deploy_url = pipeline._deployment._url()

# submit the request via POST, import as pandas DataFrame
response = requests.post(
                    deploy_url, 
                    data=singleton.to_json(orient="records"), 
                    headers=headers)

display(response.json())

[{'time': 1710513169909,
  'in': {'bathrooms': 1.0,
   'bedrooms': 3,
   'condition': 3,
   'date': '2023-08-02',
   'floors': 1.0,
   'grade': 7,
   'id': 7129300520,
   'lat': 47.5112,
   'list_price': 221900.0,
   'long': -122.257,
   'sale_price': 221900.0,
   'sqft_above': 1180,
   'sqft_basement': 0,
   'sqft_living': 1180,
   'sqft_living15': 1340,
   'sqft_lot': 5650,
   'sqft_lot15': 5650,
   'view': 0,
   'waterfront': 0,
   'yr_built': 1955,
   'yr_renovated': 0,
   'zipcode': 98178},
  'out': {'variable': [224852.0]},
  'anomaly': {'count': 0},
  'metadata': {'last_model': '{"model_name":"postprocess-step","model_sha":"c4dfec3dd259395598646ce85b8efd7811840dc726bf4915c39d862b87fc7070"}',
   'pipeline_version': '247bc22d-a702-4d04-9642-4a7766407b96',
   'elapsed': [107880, 7593739, 337280, 2178980],
   'dropped': [],
   'partition': 'engine-5dc7776c94-drhsg'}}]

In [17]:
result = pipeline.infer(singleton)
display(result.loc[:, ['time', 'out.variable']])

,time,out.variable
0,2024-03-15 14:32:27.690,[224852.0]


When finished, we undeploy the pipeline to return the resources back to the environment.

In [13]:
pipeline.undeploy()

name,housing-pipe
created,2024-03-14 21:40:49.410206+00:00
last_updated,2024-03-15 14:30:21.730581+00:00
deployed,False
arch,None
accel,None
tags,
versions,"2f010e0a-d8a4-432b-9bc0-e0aa6e99fa83, c233aaf7-565b-449b-b3bc-1ac099679663, a930f821-31aa-4adb-a29a-33044bb69259, b8d12b5c-27f7-4985-a6ba-e6c53f549018, 3c6a77cf-296d-433a-874d-0267ce544c11"
steps,preprocess-step
published,True


With this stage complete, we can proceed to Stage 4: Regular Batch Inference.

In [14]:
pub = pipeline.publish()
pub

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is publishing.......... Published.


ID,5
Pipeline Name,housing-pipe
Pipeline Version,26cfb381-ecae-465b-a9c5-15a459a0993f
Status,Published
Engine URL,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4676
Pipeline URL,ghcr.io/wallaroolabs/doc-samples/pipelines/housing-pipe:26cfb381-ecae-465b-a9c5-15a459a0993f
Helm Chart URL,oci://ghcr.io/wallaroolabs/doc-samples/charts/housing-pipe
Helm Chart Reference,ghcr.io/wallaroolabs/doc-samples/charts@sha256:453ea77cb3b2dd833536298bb75e7553be9ce5e664e78d35cd85485962a03e0f
Helm Chart Version,0.0.1-26cfb381-ecae-465b-a9c5-15a459a0993f
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 4.0, 'memory': '3Gi'}, 'requests': {'cpu': 4.0, 'memory': '3Gi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': None}, 'enginelb': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 0.2, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}}"
User Images,[]


In [20]:
# create docker run 

docker_command = f'''
docker run -p 8080:8080 \\
    -e DEBUG=true \\
    -e OCI_REGISTRY=$REGISTRYURL \\
    -e CONFIG_CPUS=1 \\
    -e OCI_USERNAME=$OCI_USERNAME \\
    -e OCI_PASSWORD=$OCI_PASSWORD \\
    -e PIPELINE_URL={pub.pipeline_url} \\
    {pub.engine_url}
'''

print(docker_command)


docker run -p 8080:8080 \
    -e DEBUG=true \
    -e OCI_REGISTRY=$REGISTRYURL \
    -e CONFIG_CPUS=1 \
    -e OCI_USERNAME=$OCI_USERNAME \
    -e OCI_PASSWORD=$OCI_PASSWORD \
    -e PIPELINE_URL=ghcr.io/wallaroolabs/doc-samples/pipelines/housing-pipe:26cfb381-ecae-465b-a9c5-15a459a0993f \
    ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4676



In [21]:
# inference via API call

import requests

# set Content-Type type
headers = {
    'Content-Type': 'application/json; format=pandas-records'
}

deploy_url = 'http://testboy.local:8080/pipelines/housing-pipe'

# submit the request via POST, import as pandas DataFrame
response = requests.post(
                    deploy_url, 
                    data=singleton.to_json(orient="records"), 
                    headers=headers)

display(response.json())

{'status': 'Error',
 'error': 'Error occurred while creating a new object: transport error: transport error'}